# Load packages and data

In [ ]:
import pandas as pd, pickle
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import gpboost
import gpboost as gpb

class GPBoostClassifier:
    def __init__(self,scan=False,use_coords=False,random_state=42,boosting_type='gbdt'):
        self.scan=scan
        self.use_coords=use_coords
        self.random_state=random_state
        self.boosting_type=boosting_type
    
    def fit(self,X,y,groups,coords=None):
        data_train = gpb.Dataset(X, y)
        self.gp_model = gpb.GPModel(group_data=groups, likelihood="bernoulli_logit", gp_coords=coords.values if self.use_coords else None,cov_function="exponential")
        params = {'learning_rate': 1e-1, 'min_data_in_leaf': 20, 'objective': "binary",
                  'verbose': 0}
        if self.boosting_type!='gbdt':
            assert self.boosting_type in ['rf','dart']
            params['boosting']=self.boosting_type
        num_boost_round = 300
        
        if self.scan:
            param_grid_small = {'learning_rate': [0.1,0.01,0.001], 'min_data_in_leaf': [20,50,100],
                                'max_depth': [5,10,15], 'max_bin': [255,1000], 'use_nesterov_acc': [False,True]}
            opt_params = gpb.grid_search_tune_parameters(param_grid=param_grid_small,
                                                         params=params,
                                                         num_try_random=15,
                                                         folds=list(GroupShuffleSplit(random_state=42).split(X,y,groups)),
                                                         gp_model=self.gp_model,
                                                         use_gp_model_for_validation=True,
                                                         train_set=data_train,
                                                         verbose_eval=1,
                                                         num_boost_round=num_boost_round,#50 
                                                         early_stopping_rounds=10,
                                                         seed=1,
                                                         metrics='binary_logloss') 

            params=opt_params['best_params']

        self.gpm = gpb.train(params=params,
                    train_set=data_train,
                    gp_model=self.gp_model,
                    num_boost_round=num_boost_round,
                   )
        return self
    
    def predict_proba(self,X,groups,coords=None):
        y_pred = self.gpm.predict(data=X, group_data_pred=groups, gp_coords_pred=coords.values if self.use_coords else None,
                            predict_var=True, raw_score=False)['response_mean']
        return y_pred

In [ ]:
data=pd.read_pickle("../../data/final_splits.pkl")

# XGBoost

In [ ]:
results=dict()
for k in data:
    results[k]=dict()
    for cv_idx in range(10):
        train_idx,test_idx=data[k]['splits'].iloc[:,cv_idx],(~data[k]['splits'].iloc[:,cv_idx])
        rf=XGBClassifier(random_state=42).fit(data[k]['X'].loc[train_idx],data[k]['y'].loc[train_idx])
        y_pred=rf.predict_proba(data[k]['X'].loc[test_idx])[:,1]
        results[k][cv_idx+1]=dict(y_pred=y_pred,
                              y_true=data[k]['y'].loc[test_idx].astype(float))
pickle.dump(results,open("../../results/xgboost.pkl",'wb'))

In [ ]:
import shap
explainers_xg=dict()
shap_vals_xg=dict()
shap_interactions_xg=dict()

for k in data:
    rf=XGBClassifier(random_state=42).fit(data[k]['X'],data[k]['y'])
    explainers_xg[k]=shap.TreeExplainer(rf)
    shap_vals_xg[k] = explainers_xg[k].shap_values(data[k]['X'])
    shap_interactions_xg[k]=explainers_xg[k].shap_interaction_values(data[k]['X'])

In [ ]:
k="mets_intra"
shap.summary_plot(shap_vals_xg[k], data[k]["X"])

In [ ]:
k="mets_away"
shap.summary_plot(shap_vals_xg[k], data[k]["X"])

In [ ]:
k="macro_ws"
shap.summary_plot(shap_vals_xg[k], data[k]["X"])

In [ ]:
k="mets_overall"
shap.dependence_plot("CD20", shap_vals_xg[k], data[k]["X"])

# RF

In [ ]:
results=dict()
for k in data:
    results[k]=dict()
    for cv_idx in range(10):
        train_idx,test_idx=data[k]['splits'].iloc[:,cv_idx],(~data[k]['splits'].iloc[:,cv_idx])
        rf=RandomForestClassifier(random_state=42).fit(data[k]['X'].loc[train_idx],data[k]['y'].loc[train_idx])
        y_pred=rf.predict_proba(data[k]['X'].loc[test_idx])[:,1]
        results[k][cv_idx+1]=dict(y_pred=y_pred,
                              y_true=data[k]['y'].loc[test_idx].astype(float))
pickle.dump(results,open("../../results/rf.pkl",'wb'))

# GPBoost

In [ ]:
import numpy as np
results=dict()
for k in data:
    results[k]=dict()
    for cv_idx in range(10):
        np.random.seed(42)
        train_idx,test_idx=data[k]['splits'].iloc[:,cv_idx],(~data[k]['splits'].iloc[:,cv_idx])
        gpc=GPBoostClassifier(random_state=42).fit(data[k]['X'].loc[train_idx],data[k]['y'].loc[train_idx],data[k]['group'].loc[train_idx])
        y_pred=gpc.predict_proba(data[k]['X'].loc[test_idx],data[k]['group'].loc[test_idx])
        results[k][cv_idx+1]=dict(y_pred=y_pred,
                              y_true=data[k]['y'].loc[test_idx].astype(float))
pickle.dump(results,open("../../results/gpb.pkl",'wb'))

In [ ]:
explainers=dict()
shap_vals=dict()
shap_interactions=dict()

for k in data:
    gpc=GPBoostClassifier(random_state=42).fit(data[k]['X'],data[k]['y'],data[k]['group'])
    explainers[k]=shap.TreeExplainer(gpc.gpm)
    shap_vals[k] = explainers[k].shap_values(data[k]['X'],data[k]['group'])
    shap_interactions[k]=explainers[k].shap_interaction_values(data[k]['X'],data[k]['group'])

In [ ]:
k="macro_ws"
shap.dependence_plot("CTLA4", shap_vals[k], data[k]["X"])

In [ ]:
k="mets_intra"
shap.summary_plot(shap_vals[k], data[k]["X"])

In [ ]:
k="mets_overall"
shap.summary_plot(shap_vals[k], data[k]["X"])

In [ ]:
k="macro_ws"
shap.summary_plot(shap_vals[k], data[k]["X"], )

In [ ]:
import matplotlib,matplotlib.pyplot as plt
matplotlib.rcParams['figure.dpi']=300
k="mets_overall"
shap.dependence_plot("CD20", shap_vals[k], data[k]["X"])

In [ ]:
k="mets_intra"
shap.dependence_plot("CD8", shap_vals[k], data[k]["X"], interaction_index="CD45")

In [ ]:
k="mets_inter"
shap.dependence_plot("CTLA4", shap_vals[k], data[k]["X"], interaction_index="CD68")

In [ ]:
k="macro_ws"
shap.dependence_plot("CD68", shap_vals[k], data[k]["X"], interaction_index="PanCk")

In [ ]:
k="macro_ws"
shap.dependence_plot("CD34", shap_vals[k], data[k]["X"], interaction_index="PanCk")

In [ ]:
shap.dependence_plot("Beta_2_microglobulin", shap_vals[k], data[k]["X"])

In [ ]:
k="mets_inter"
shap.dependence_plot("CD68", shap_vals[k], data[k]["X"])

In [ ]:
results=dict()
for k in data:
    if "macro" in k:
        if k not in list(results.keys()): results[k]=dict()
        for cv_idx in range(10):
            np.random.seed(42)
            if (cv_idx+1) not in list(results[k].keys()):
                print(k,cv_idx+1)
                train_idx,test_idx=data[k]['splits'].iloc[:,cv_idx],(~data[k]['splits'].iloc[:,cv_idx])
                rf=GPBoostClassifier(random_state=42,use_coords=True).fit(data[k]['X'].loc[train_idx],data[k]['y'].loc[train_idx],data[k]['group'].loc[train_idx],coords=data[k]['coords'].loc[train_idx])
                y_pred=rf.predict_proba(data[k]['X'].loc[test_idx],data[k]['group'].loc[test_idx],coords=data[k]['coords'].loc[test_idx])
                results[k][cv_idx+1]=dict(y_pred=y_pred,
                                      y_true=data[k]['y'].loc[test_idx].astype(float))
pickle.dump(results,open("../../results/gpb_coords.pkl",'wb'))

# Interactions

In [ ]:
import shap

interactions_dict=dict()
for k in data:
    interactions_dict[k]=dict()
    for cv_idx in range(10):
        print(k,cv_idx)
        np.random.seed(42)
        train_idx,test_idx=data[k]['splits'].iloc[:,cv_idx],(~data[k]['splits'].iloc[:,cv_idx])
        gpc=GPBoostClassifier(random_state=42).fit(data[k]['X'].loc[train_idx],data[k]['y'].loc[train_idx],data[k]['group'].loc[train_idx])
        explainer=shap.TreeExplainer(gpc.gpm)
        shap_values = explainer.shap_values(data[k]['X'].loc[train_idx],data[k]['group'].loc[train_idx])
        interaction_val=explainer.shap_interaction_values(data[k]['X'].loc[train_idx])
        interactions=np.abs(interaction_val).sum(0)
        interactions[np.eye(*interactions.shape).astype(bool)]=0
        interactions=pd.DataFrame(interactions,columns=data[k]['X'].columns,index=data[k]['X'].columns)
        interactions_dict[k][cv_idx+1]=interactions
pickle.dump(interactions_dict,open('../../interactions/interactions.pkl','wb'))

# Output GPBoost interactions to R BGLMM

In [ ]:
from functools import reduce
interactions_dict=pd.read_pickle('../../interactions/interactions.pkl')

In [ ]:
import numpy as np
for k in data:
    all_interaction_shap_scores=reduce(lambda x,y:x+y,list(interactions_dict[k].values())).where(np.triu(np.ones(interactions_dict[k][1].shape),k=1).astype(np.bool)).stack().reset_index()
    all_interaction_shap_scores.columns=['feature_1','feature_2', 'shap_interaction_score']
    all_interaction_shap_scores=all_interaction_shap_scores.sort_values('shap_interaction_score',ascending=False)
    print(k,'      ','+'.join([':'.join(x) for x in all_interaction_shap_scores.iloc[:10,:2].values.tolist()]))


In [ ]:
shap_vals_df=dict()
for k in data:
    all_interaction_shap_scores=reduce(lambda x,y:x+y,list(interactions_dict[k].values())).where(np.triu(np.ones(interactions_dict[k][1].shape),k=1).astype(np.bool)).stack().reset_index()
    all_interaction_shap_scores.columns=['feature_1','feature_2', 'shap_interaction_score']
    shap_vals_df[k]=all_interaction_shap_scores.sort_values('shap_interaction_score',ascending=False).iloc[:10,:2].apply(lambda x: ":".join(x),axis=1).reset_index(drop=True)